In [ ]:
#1a - the weight of observation is 1/100 which is equal is .01
#1b - the weight of the 72nd observation will be larger. The model will learn from its previous errors/mistakes
#2 - since weak models are added and trained using weighted training data, this is why adaboost.m1 is considered
#an ensemble learning algorithm. Ensemble learnig is a general meta approach to machine learning that seeks better
#predictive performance by combining the predictions from multiple models.
#4 - I would increase the number of estimators
#5 - A, B, C

In [28]:
import boto3 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler

#defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [29]:
#input and target variables
X = heart[['age', 'totChol','sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

#splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)

In [30]:
#transforming inputs to 0-1 scale
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [31]:
#random forest
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

#predicting test data
pred1 = rf_md.predict_proba(X_test)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1)

#computing the recall
recall_score(Y_test, pred1)

0.9107142857142857

In [32]:
#adaboost model
ab_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

#predicting test data
pred2 = ab_md.predict_proba(X_test)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1)

#computing the recall
recall_score(Y_test, pred2)

1.0

In [33]:
from sklearn.svm import SVC

#adaboost with 50 learners and svc as learner
ab_svc = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 50).fit(X_train, Y_train)

#predicting test data
ab_svc_preds = ab_svc.predict_proba(X_test)[:, 1]
ab_svc_preds = np.where(ab_svc_preds < 0.10, 0, 1)

#computing the recall
recall_score(Y_test, ab_svc_preds)

1.0

In [ ]:
final_md1 = []
final_md2 = []
final_md3 = []

#repeating 100 times
for i in range(0, 100):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)
    
    rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    ab_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)
    ab_svc = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 50).fit(X_train, Y_train)
    
    pred1 = rf_md.predict_proba(X_test)[:, 1]
    pred1 = np.where(pred1 < .1, 0, 1)

    pred2 = ab_md.predict_proba(X_test)[:, 1]
    pred2 = np.where(pred2 < .1, 0, 1)

    ab_svc_preds = ab_svc.predict_proba(X_test)[:, 1]
    ab_svc_preds = np.where(ab_svc_preds < .1, 0, 1)
   
    final_md1.append(recall_score(Y_test, pred1))
    final_md2.append(recall_score(Y_test, pred2))
    final_md3.append(recall_score(Y_test, ab_svc_preds))
    
print('Model 1 avg recall score is:', np.mean(final_md1))
print('Model 2 avg recall score is:', np.mean(final_md2))
print('Model 3 avg recall score is:', np.mean(final_md3))

In [ ]:
#based on the models, I would choose the random forest